Importing Libraries to Start

In [2]:
from bs4 import BeautifulSoup
import requests
import numpy as np
import pandas as pd
import regex as re

# Function Definition

Guidance on Web Scraping with Beautiful Soup
https://github.com/brodriguezmilla/WebScrapingCFCBS4/blob/master/scrapingMailingAddresses-shortExplained.ipynb

Getting Unique Values for List
https://www.geeksforgeeks.org/python-get-unique-values-list/


In [12]:
def link_getting(url, my_headers, explore = True, html_get = "a", reg_exp = ""):

    assert isinstance(url, str), f"url must be a String"
    assert isinstance(reg_exp, str), f"Regular Expression must be a String"

    session = requests.Session()
    try:
        con = session.get(url, headers = my_headers)
    except Exception as e:
        print("Error feteching the URL ", url)
        print(e)
    try:
        soup = BeautifulSoup(con.text, "html.parser")
    except Exception as e:
        print("Could not parse: ", url)
        print(e)
    if explore:
        container_list = soup.find_all(html_get)
        return([tag for tag in container_list])
    else:
        container_list = soup.find_all(html_get, href=re.compile(reg_exp))
        return([tag.get("href") for tag in container_list])
    
def unique_array(list1):

    assert isinstance(list1, list), f"List1 must be a list"

    temp = np.array(list1)
    return np.unique(temp)

def link_cleaning(list1, reg_exp = ""):

    assert isinstance(list1, list), f"List1 must be a list"
    assert isinstance(reg_exp, str), f"Regular Expression must be a String"

    temp = np.char.strip(unique_array(list1))
    regex = re.compile(reg_exp)
    return [tag for tag in temp if regex.match(tag)]

# Defining the Website Start

In [4]:
my_headers = {"User-Agent": "Mozilla/5.0 (X11; Linux x86_64; rv:107.0) Gecko/20100101 Firefox/107.0"}

brookings = "https://www.brookings.edu/topics"

heritage = "https://www.heritage.org"

brookings_topic_regex = ".+(topic)+/"
brookings_blog_regex = ".+(blog|research)+/"

heritage_regex = "^/\w*+$"
heritage_commentary_regex = ".+(commentary)+/"

## Brookings Link Gathering

In [5]:
brookings_initial_list = link_getting(brookings, my_headers, explore = False, html_get = "a", reg_exp = "")

In [40]:
brookings_topic_initial = link_cleaning(brookings_initial_list, brookings_topic_regex)

## Brookings Test

In [41]:
brookings_test = link_getting(brookings_topic_initial[23], my_headers, explore = False, reg_exp=brookings_blog_regex)

In [50]:
brookings_iterated_list = []

for tag in brookings_topic_initial:
    tag_list = link_getting(tag, my_headers, explore = False, reg_exp=brookings_blog_regex)
    for sub_tag in tag_list:
        brookings_iterated_list.append(sub_tag)

In [51]:
brookings_final = link_cleaning(brookings_iterated_list)

# Heritage Link Pulls

In [6]:
heritage_initial_list = link_getting(heritage, my_headers, explore = False, html_get = "a", reg_exp = "^/\w*")

In [43]:
heritage_topic_initial = link_cleaning(heritage_initial_list, heritage_regex)
heritage_topic_initial = heritage_topic_initial[1:]
heritage_topic_initial = [heritage+tag for tag in heritage_topic_initial]

In [46]:
heritage_test = link_getting(heritage_topic_initial[1], my_headers, explore = False, reg_exp=heritage_commentary_regex)

In [47]:
heritage_iterated_list = []

for tag in heritage_topic_initial:
    tag_list = link_getting(tag, my_headers, explore = False, reg_exp=heritage_commentary_regex)
    for sub_tag in tag_list:
        heritage_iterated_list.append(sub_tag)

In [48]:
heritage_final = []
heritage_link_check_regex = re.compile(heritage)
for tag in heritage_iterated_list:
    if heritage_link_check_regex.match(tag):
        heritage_final.append(tag)
    else:
        heritage_final.append(heritage + tag)


In [49]:
heritage_final = link_cleaning(heritage_final)